In [ ]:
import ee
import geemap
import joblib
import numpy as np
import os
import threading
from threading import Lock

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Load RandomForest model
model = joblib.load('models/cacu/clorofila/model.joblib')
scaler = joblib.load('models/cacu/clorofila/scaler.joblib')

# Define area of interest
aoi = ee.Geometry.Polygon([[[-45.559114, -18.954365], [-45.559114, -18.212409], 
                           [-44.839706, -18.212409], [-44.839706, -18.954365], 
                           [-45.559114, -18.954365]]])

# Function to process a single tile
def process_tile(tile_geometry, date_range, model, scaler):
    # Sentinel-2 collection for tile
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(tile_geometry) \
        .filterDate(date_range[0], date_range[1]) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    # Get median image
    image = sentinel2.median().clip(tile_geometry)
    
    # Select bands of interest
    bands = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11']
    image = image.select(bands)
    
    # Apply water mask
    MNDWI = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    water_mask = MNDWI.gt(0.3)
    image = image.updateMask(water_mask)
    
    # Calculate indices
    NDCI = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
    NDVI = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    FAI = image.expression(
        'NIR - (RED + (SWIR - RED) * (NIR_wl - RED_wl) / (SWIR_wl - RED_wl))',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'SWIR': image.select('B11'),
            'NIR_wl': 842,
            'RED_wl': 665,
            'SWIR_wl': 1610
        }
    ).rename('FAI')
    
    # Calculate band ratios
    B3_B2_ratio = image.select('B3').divide(image.select('B2')).rename('B3_B2_ratio')
    B4_B3_ratio = image.select('B4').divide(image.select('B3')).rename('B4_B3_ratio')
    B5_B4_ratio = image.select('B5').divide(image.select('B4')).rename('B5_B4_ratio')
    
    # Get date information
    middle_date = ee.Date(sentinel2.limit(1).first().get('system:time_start'))
    month = ee.Image.constant(middle_date.get('month')).rename('Month')
    season = ee.Image.constant(middle_date.get('month').add(2).divide(3).floor().add(1)).rename('Season')
    
    # Combine all features
    image_with_indices = image.addBands([NDCI, NDVI, FAI, B3_B2_ratio, B4_B3_ratio, 
                                       B5_B4_ratio, month, season])
    
    # Create scaled bands
    feature_names = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'NDCI', 'NDVI', 'FAI', 
                    'B3_B2_ratio', 'B4_B3_ratio', 'B5_B4_ratio', 'Month', 'Season']
    
    scaled_bands = []
    for i, name in enumerate(feature_names):
        scaled_band = image_with_indices.select(name) \
            .subtract(ee.Number(scaler.mean_[i])) \
            .divide(ee.Number(scaler.scale_[i])) \
            .rename(f'scaled_{name}')
        scaled_bands.append(scaled_band)
    
    # Combine scaled bands
    scaled_image = ee.Image.cat(scaled_bands)
    
    # Create prediction
    weighted_bands = []
    for i, (name, importance) in enumerate(zip(feature_names, model.feature_importances_)):
        weighted_band = scaled_image.select(f'scaled_{name}').multiply(ee.Number(importance))
        weighted_bands.append(weighted_band)
    
    predicted_image = ee.Image.cat(weighted_bands).reduce(ee.Reducer.sum()).rename('chlorophyll_pred')
    
    return predicted_image.updateMask(water_mask)

# Function to split AOI and process tiles in parallel
def process_aoi_in_tiles(aoi, n_tiles, date_range, model, scaler, save_directory):
    os.makedirs(save_directory, exist_ok=True)
    
    # Get AOI bounds
    aoi_bounds = aoi.bounds().coordinates().getInfo()[0]
    xmin, ymin = aoi_bounds[0][0], aoi_bounds[0][1]
    xmax, ymax = aoi_bounds[2][0], aoi_bounds[2][1]
    
    # Calculate tile sizes
    x_step = (xmax - xmin) / n_tiles
    y_step = (ymax - ymin) / n_tiles
    
    lock = Lock()
    tile_results = []
    
    def process_and_save_tile(i, j):
        # Create tile geometry
        x0 = xmin + i * x_step
        x1 = xmin + (i + 1) * x_step
        y0 = ymin + j * y_step
        y1 = ymin + (j + 1) * y_step
        tile_geometry = ee.Geometry.Polygon([[[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]])
        
        # Process tile
        tile_result = process_tile(tile_geometry, date_range, model, scaler)
        
        # Save tile result
        out_file = os.path.join(save_directory, f'PredictedChlorophyll_Tile_{i+1}_{j+1}.tif')
        with lock:
            try:
                geemap.ee_export_image(
                    tile_result,
                    filename=out_file,
                    scale=30,
                    region=tile_geometry
                )
                print(f'Tile {i+1}_{j+1} processed and saved: {out_file}')
                tile_results.append(tile_result)
            except Exception as e:
                print(f"Error processing tile {i+1}_{j+1}: {str(e)}")
    
    # Create and start threads for each tile
    threads = []
    for i in range(n_tiles):
        for j in range(n_tiles):
            t = threading.Thread(target=process_and_save_tile, args=(i, j))
            threads.append(t)
            t.start()
    
    # Wait for all threads to complete
    for t in threads:
        t.join()
    
    # Merge all tiles
    merged_result = ee.ImageCollection(tile_results).mosaic()
    return merged_result

# Main execution
date_range = ['2020-01-01', '2020-04-01']
save_directory = 'analises_clorofila/cacu'
n_tiles = 4  # NxN grid

# Process the entire AOI in tiles
final_result = process_aoi_in_tiles(aoi, n_tiles, date_range, model, scaler, save_directory)

# Calculate min and max values
min_max_values = final_result.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['chlorophyll_pred_min']
max_value = min_max_values['chlorophyll_pred_max']

# Display results
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        'blue', 'cyan', 'green', 'yellow', 'orange', 'red',
        'darkred', 'purple', 'magenta', 'brown', 'black'
    ]
}

Map.addLayer(final_result, vis_params, 'Predicted Chlorophyll')
Map.addLayer(aoi, {'color': 'white', 'width': 2, 'fillColor': 'transparent'}, 'AOI Boundary')
Map.addLayerControl()

def add_legend(map_obj, title, palette, min_value, max_value):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>{min_value:.2f}</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>{max_value:.2f}</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

add_legend(Map, 'Predicted Chlorophyll', vis_params['palette'], min_value, max_value)

Map

Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\water_quality_maps\analises_clorofila\cacu\PredictedChlorophyll_Tile_1_1.tif
Tile 1_1 processed and saved: analises_clorofila/cacu\PredictedChlorophyll_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\water_quality_maps\analises_clorofila\cacu\PredictedChlorophyll_Tile_1_2.tif
Tile 1_2 processed and saved: analises_clorofila/cacu\PredictedChlorophyll_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\water_quality_maps\analises_clorofila\cacu\PredictedChlorophyll_Tile_2_1.tif
Tile 2_1 processed and saved: analises_clorofila/cacu\PredictedChlorophyll_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\water_quality_maps\analises_clorofila\cacu\PredictedChlorophyll_Tile_2_2.tif
Tile 2_2 processed and saved: analises_clorofila/cacu\PredictedChlorophyll_Tile_2_2.tif


Map(center=[-18.58345884758661, -45.1994100000001], controls=(WidgetControl(options=['position', 'transparent_…